<a href="https://colab.research.google.com/github/thaiphi/Stock-Markets-Analytics-2024/blob/main/Week%202/Module_2_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install yfinance

In [3]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt


##Question 1: IPO Filings Web Scraping and Data Processing
What's the total sum ($m) of 2023 filings that happenned of Fridays?

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/


1. Convert the 'Filing Date' to datetime()
2. 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs)
3. Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given)You may be inspired by the function extract_numbers() in [Code Snippet 4], or you can write your own function to "parse" a string.
4. Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)
5. Find the total sum in $m (millions of USD, closest INTEGER number) for all fillings during 2023, which happened on Fridays (Date.dt.dayofweek()==4). You should see 32 records in total, 24 of it is not null.

(additional: you can read about S-1 IPO filing to understand the context)

In [ ]:
import pandas as pd
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [ ]:
ipo_fillings = ipo_dfs[0]

ipo_fillings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326 entries, 0 to 325
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     326 non-null    object
 1   Symbol          326 non-null    object
 2   Company Name    326 non-null    object
 3   Price Range     326 non-null    object
 4   Shares Offered  326 non-null    object
dtypes: object(5)
memory usage: 12.9+ KB


In [ ]:
ipo_fillings.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
1,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-
2,"Apr 22, 2024",DRJT,Derun Group Inc,$5.00,-
3,"Apr 19, 2024",GPAT,GP-Act III Acquisition Corp.,$10.00,25000000
4,"Apr 16, 2024",JLJT,Jialiang Holdings Ltd,$5.00,-


##Cleaning the data

In [ ]:
# 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs)

import numpy as np
ipo_fillings['Shares Offered'] = ipo_fillings['Shares Offered'].apply(lambda x: float(x) if x != '-' else np.nan)


In [ ]:
# Convert 'Filing Date' to datetime
ipo_fillings['Filing Date'] = pd.to_datetime(ipo_fillings['Filing Date'])


In [ ]:
# Problem --> not always the columns are filled
missing_prices_df = ipo_fillings[ipo_fillings['Price Range'].astype(str).str.find('-') >= 0]
missing_prices_df

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
1,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN
5,2024-04-16,GGL,Games Global Limited,-,NaN
6,2024-04-15,GAUZ,Gauzy Ltd.,-,NaN
7,2024-04-12,BOW,Bowhead Specialty Holdings Inc.,-,NaN
8,2024-04-05,SPHL,Springview Holdings Ltd,$4.00 - $5.00,2000000.0
...,...,...,...,...,...
319,2020-06-17,ALEH,ALE Group Holding Limited,$4.00 - $6.00,4200000.0
320,2020-02-18,CAST,"FreeCast, Inc.",-,NaN
321,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0
323,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0


In [ ]:
#Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified,
#to the price (if only one number is provided), or to the average of 2 prices (if a range is given)
def str_to_digits(s):
    if not s or s == '-':  # If empty string or just a dash
        return np.nan
    s = s.replace('$', '').replace(',', '')  # Remove $ and commas
    if '-' in s:  # If it's a range
        try:
            start, end = map(float, s.split('-'))
            return (start + end) / 2  # Return average of the range
        except ValueError:
            return np.nan  # Return nan if conversion fails
    else:
        try:
            return float(s)  # Convert to float
        except ValueError:
            return np.nan  # Return nan if conversion fails

# Apply the function to the 'Price Range' column
ipo_fillings['Avg_price'] = ipo_fillings['Price Range'].apply(lambda x: str_to_digits(x))

print(ipo_fillings)

    Filing Date Symbol                           Company Name     Price Range  \
0    2024-04-26   EURK                Eureka Acquisition Corp          $10.00   
1    2024-04-26    HDL    Super Hi International Holding Ltd.               -   
2    2024-04-22   DRJT                        Derun Group Inc           $5.00   
3    2024-04-19   GPAT           GP-Act III Acquisition Corp.          $10.00   
4    2024-04-16   JLJT                  Jialiang Holdings Ltd           $5.00   
..          ...    ...                                    ...             ...   
321  2020-01-21   GOXS                            Goxus, Inc.  $8.00 - $10.00   
322  2020-01-21   UTXO                 UTXO Acquisition, Inc.          $10.00   
323  2019-12-09   LOHA                           Loha Co. Ltd  $8.00 - $10.00   
324  2019-10-04   ZGHB  China Eco-Materials Group Co. Limited           $4.00   
325  2018-12-27   FBOX              Fit Boxx Holdings Limited   $5.00 - $6.50   

     Shares Offered  Avg_pr

In [ ]:
# Define a column "Shares_offered_value",
# which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)
def calculate_shares_offered_value(row):
    shares_offered = row['Shares Offered']
    avg_price = row['Avg_price']
    if pd.notnull(shares_offered) and pd.notnull(avg_price):
        return shares_offered * avg_price
    else:
        return np.nan

# Apply the function to create the Shares_offered_value column
ipo_fillings['Shares_offered_value'] = ipo_fillings.apply(calculate_shares_offered_value, axis=1)

print(ipo_fillings)

    Filing Date Symbol                           Company Name     Price Range  \
0    2024-04-26   EURK                Eureka Acquisition Corp          $10.00   
1    2024-04-26    HDL    Super Hi International Holding Ltd.               -   
2    2024-04-22   DRJT                        Derun Group Inc           $5.00   
3    2024-04-19   GPAT           GP-Act III Acquisition Corp.          $10.00   
4    2024-04-16   JLJT                  Jialiang Holdings Ltd           $5.00   
..          ...    ...                                    ...             ...   
321  2020-01-21   GOXS                            Goxus, Inc.  $8.00 - $10.00   
322  2020-01-21   UTXO                 UTXO Acquisition, Inc.          $10.00   
323  2019-12-09   LOHA                           Loha Co. Ltd  $8.00 - $10.00   
324  2019-10-04   ZGHB  China Eco-Materials Group Co. Limited           $4.00   
325  2018-12-27   FBOX              Fit Boxx Holdings Limited   $5.00 - $6.50   

     Shares Offered  Avg_pr

In [ ]:
# Find the total sum in $m (millions of USD, closest INTEGER number) for all fillings during 2023,
# which happened on Fridays (Date.dt.dayofweek()==4). You should see 32 records in total, 24 of it is not null.
friday_fillings = ipo_fillings[(ipo_fillings['Filing Date'].dt.dayofweek == 4) & (ipo_fillings['Filing Date'].dt.year == 2023)]
total_sum_in_millions = int(friday_fillings['Shares_offered_value'].sum() / 1000000)
print(f"Total sum of 2023 filings that happened on Fridays: ${total_sum_in_millions}m")


Total sum of 2023 filings that happened on Fridays: $285m


##Question 2: IPOs "Fixed days hold" strategy
Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?

Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/). Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks. Some of the tickers like 'DYCQ' and 'LEGT' were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day). Find X, when the 75% quantile growth (among 185 investments) is the highest.

HINTs:

You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
You can use the DataFrame.describe() function to get mean, min, max, 25-50-75% quantiles.
Addtional:

You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
What's your recommendation: Do you suggest pursuing this strategy for an optimal X?


In [ ]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [ ]:
ipos_2023 = ipo_dfs[0]
ipos_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      154 non-null    object
 1   Symbol        154 non-null    object
 2   Company Name  154 non-null    object
 3   IPO Price     154 non-null    object
 4   Current       154 non-null    object
 5   Return        154 non-null    object
dtypes: object(6)
memory usage: 7.3+ KB


In [ ]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [ ]:
ipos_2024 = ipo_dfs[0]
ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      64 non-null     object
 1   Symbol        64 non-null     object
 2   Company Name  64 non-null     object
 3   IPO Price     64 non-null     object
 4   Current       64 non-null     object
 5   Return        64 non-null     object
dtypes: object(6)
memory usage: 3.1+ KB


In [ ]:
# "stacking dataframes"
# pd.concat() is used to concatenate the DataFrames vertically.
# The ignore_index=True parameter ensures that the resulting DataFrame has a new index,
# ignoring the original indices of the input DataFrames.
# The stacked_df now contains the concatenated DataFrame.
stacked_ipos_df = pd.concat([ipos_2024, ipos_2023], ignore_index=True)

In [ ]:
stacked_ipos_df.head(1)

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 1, 2024",VIK,Viking Holdings Ltd.,$24.00,$26.10,8.75%


In [ ]:
# Correctly applied transformations with 'defensive' techniques, but now not all are non-null
stacked_ipos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      218 non-null    object
 1   Symbol        218 non-null    object
 2   Company Name  218 non-null    object
 3   IPO Price     218 non-null    object
 4   Current       218 non-null    object
 5   Return        218 non-null    object
dtypes: object(6)
memory usage: 10.3+ KB


In [ ]:

# Problem --> not always the columns are filled
missing_prices_df = stacked_ipos_df[stacked_ipos_df['IPO Price'].astype(str).str.find('-') >= 0]
missing_prices_df

,IPO Date,Symbol,Company Name,IPO Price,Current,Return


In [ ]:
# it has some missing values --> use defensive errors='coerce' (if don't have time to crack into the data errors)
#     : pd.to_numeric() function call, which will convert problematic values to NaN.
#     otherwise you'll get a ValueError: Unable to parse string "-" at position 9
stacked_ipos_df['IPO Price'] = pd.to_numeric(stacked_ipos_df['IPO Price'].str.replace('$', ''), errors='coerce')
# not sure why, but need to call it again to transform 'object' to 'float64'
stacked_ipos_df['IPO Price'] = pd.to_numeric(stacked_ipos_df['IPO Price'])

In [ ]:
# Convert "Current" column
stacked_ipos_df['Current'] = pd.to_numeric(stacked_ipos_df['Current'].str.replace('$', ''), errors='coerce')

# Convert 'Return' to numeric format (percentage)
stacked_ipos_df['Return'] = pd.to_numeric(stacked_ipos_df['Return'].str.replace('%', ''), errors='coerce') / 100

In [ ]:
# simple way of checking NULLs
# (you need to understand how vector operations work .isnull() and calls chaining .isnull().sum())
stacked_ipos_df.isnull().sum()

IPO Date        0
Symbol          0
Company Name    0
IPO Price       0
Current         0
Return          1
dtype: int64

In [ ]:
# Do you want to leave the record or not?
stacked_ipos_df[stacked_ipos_df.Return.isnull()]

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
4,"Apr 25, 2024",MRX,Marex Group plc,19.0,19.0,NaN


In [ ]:
# now you can operate with columns as a numeric type
stacked_ipos_df['IPO Price'].mean()

11.060229357798166

In [ ]:
print(stacked_ipos_df)

         IPO Date Symbol                             Company Name  IPO Price  \
0     May 1, 2024    VIK                     Viking Holdings Ltd.       24.0   
1    Apr 26, 2024   ZONE                CleanCore Solutions, Inc.        4.0   
2    Apr 25, 2024   RBRK                             Rubrik, Inc.       32.0   
3    Apr 25, 2024   LOAR                       Loar Holdings Inc.       28.0   
4    Apr 25, 2024    MRX                          Marex Group plc       19.0   
..            ...    ...                                      ...        ...   
213  Jan 25, 2023    QSG                     QuantaSing Group Ltd       12.5   
214  Jan 20, 2023   CVKD              Cadrenal Therapeutics, Inc.        5.0   
215  Jan 13, 2023   SKWD  Skyward Specialty Insurance Group, Inc.       15.0   
216  Jan 13, 2023   ISRL                 Israel Acquisitions Corp       10.0   
217  Jan 13, 2023   MGOL                          MGO Global Inc.        5.0   

     Current  Return  
0      26.10  0.

In [ ]:
# Attempt to convert 'IPO Date' to datetime using different format strings
formats = ["%b %d, %Y", "%Y-%m-%d", "%m/%d/%Y", "%B %d, %Y"]
for fmt in formats:
    try:
        stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'], format=fmt)
        break  # Stop looping if conversion succeeds
    except ValueError:
        continue  # Continue to the next format if conversion fails

print(stacked_ipos_df)

      IPO Date Symbol                             Company Name  IPO Price  \
0   2024-05-01    VIK                     Viking Holdings Ltd.       24.0   
1   2024-04-26   ZONE                CleanCore Solutions, Inc.        4.0   
2   2024-04-25   RBRK                             Rubrik, Inc.       32.0   
3   2024-04-25   LOAR                       Loar Holdings Inc.       28.0   
4   2024-04-25    MRX                          Marex Group plc       19.0   
..         ...    ...                                      ...        ...   
213 2023-01-25    QSG                     QuantaSing Group Ltd       12.5   
214 2023-01-20   CVKD              Cadrenal Therapeutics, Inc.        5.0   
215 2023-01-13   SKWD  Skyward Specialty Insurance Group, Inc.       15.0   
216 2023-01-13   ISRL                 Israel Acquisitions Corp       10.0   
217 2023-01-13   MGOL                          MGO Global Inc.        5.0   

     Current  Return  
0      26.10  0.0875  
1       3.11 -0.2225  
2     

In [ ]:
# Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01")
# - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

ipo_filter = stacked_ipos_df[stacked_ipos_df['IPO Date'] <= '2024-03-01']
ipo_filter = ipo_filter[ipo_filter.Symbol != 'RYZB']
ipo_filter

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
33,2024-02-27,SMXT,"SolarMax Technology, Inc.",4.00,10.34,1.5850
34,2024-02-22,VHAI,Vocodia Holdings Corp,4.25,0.17,-0.9602
35,2024-02-21,DYCQ,DT Cloud Acquisition Corporation,10.00,10.33,0.0170
36,2024-02-16,CHRO,Chromocell Therapeutics Corp,6.00,1.77,-0.7050
37,2024-02-14,UMAC,"Unusual Machines, Inc.",4.00,1.21,-0.6975
...,...,...,...,...,...,...
213,2023-01-25,QSG,QuantaSing Group Ltd,12.50,3.13,-0.7496
214,2023-01-20,CVKD,"Cadrenal Therapeutics, Inc.",5.00,0.45,-0.9091
215,2023-01-13,SKWD,"Skyward Specialty Insurance Group, Inc.",15.00,35.50,1.3667
216,2023-01-13,ISRL,Israel Acquisitions Corp,10.00,10.93,0.0925


In [ ]:
symbols = ipo_filter['Symbol'].tolist()
symbols[symbols.index('PTHR')] = 'PTHRF'
symbols

['SMXT',
 'VHAI',
 'DYCQ',
 'CHRO',
 'UMAC',
 'TBBB',
 'MGX',
 'HLXB',
 'TELO',
 'KYTX',
 'PMNT',
 'AHR',
 'LEGT',
 'ANRO',
 'GUTS',
 'AS',
 'FBLG',
 'BTSG',
 'AVBP',
 'HAO',
 'CGON',
 'YIBO',
 'SUGP',
 'JL',
 'KSPI',
 'JVSA',
 'PSBD',
 'CCTG',
 'SYNX',
 'SDHC',
 'ROMA',
 'IROH',
 'LGCB',
 'ZKH',
 'BAYA',
 'INHD',
 'AFJK',
 'GSIW',
 'FEBO',
 'CLBR',
 'ELAB',
 'RR',
 'DDC',
 'SHIM',
 'GLAC',
 'SGN',
 'HG',
 'CRGX',
 'ANSC',
 'AITR',
 'GVH',
 'LXEO',
 'PAPL',
 'ATGL',
 'MNR',
 'WBUY',
 'NCL',
 'BIRK',
 'GMM',
 'PMEC',
 'LRHC',
 'GPAK',
 'SPKL',
 'QETA',
 'MSS',
 'ANL',
 'SYRA',
 'VSME',
 'LRE',
 'TURB',
 'MDBH',
 'KVYO',
 'CART',
 'DTCK',
 'NMRA',
 'ARM',
 'SPPL',
 'NWGL',
 'SWIN',
 'IVP',
 'NNAG',
 'SRM',
 'SPGC',
 'LQR',
 'NRXS',
 'FTEL',
 'MIRA',
 'PXDT',
 'CTNT',
 'HRYU',
 'SRFM',
 'PRZO',
 'HYAC',
 'KVAC',
 'JNVR',
 'ELWS',
 'WRNT',
 'TSBX',
 'ODD',
 'APGE',
 'NETD',
 'SGMT',
 'BOWN',
 'SXTP',
 'PWM',
 'VTMX',
 'INTS',
 'SVV',
 'KGS',
 'FIHL',
 'GENK',
 'BUJA',
 'BOF',
 'AZTR',
 'CA

In [ ]:
# Get OHLCV daily prices for all stocks
import yfinance as yf

prices = {}
for i in symbols:
    try:
        prices[i] = yf.download(i, start='2023-01-01', interval = "1d")
    except:
        pass

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [ ]:
# Create a list of DataFrames for each stock
data_frames = []
for symbol, data in prices.items():
    # Create a DataFrame for each stock
    df_stock = pd.DataFrame(data)
    df_stock['Symbol'] = symbol  # Add a 'Symbol' column
    data_frames.append(df_stock)

# Concatenate the DataFrames into a single DataFrame
df = pd.concat(data_frames)
df.reset_index(inplace=True)
# Display the DataFrame info
print(df)

            Date   Open   High    Low  Close  Adj Close   Volume Symbol
0     2024-02-27  3.500  9.350  3.500  8.000      8.000  2840400   SMXT
1     2024-02-28  6.420  7.240  5.600  5.860      5.860   243200   SMXT
2     2024-02-29  5.740  5.840  4.300  4.890      4.890   130000   SMXT
3     2024-03-01  3.940  5.150  3.800  4.880      4.880   485000   SMXT
4     2024-03-04  4.730  5.100  4.258  4.660      4.660   114300   SMXT
...          ...    ...    ...    ...    ...        ...      ...    ...
33807 2024-04-25  0.221  0.250  0.215  0.235      0.235   278700   MGOL
33808 2024-04-26  0.225  0.232  0.216  0.221      0.221    31300   MGOL
33809 2024-04-29  0.235  0.248  0.224  0.240      0.240   128300   MGOL
33810 2024-04-30  0.241  0.260  0.222  0.236      0.236   202100   MGOL
33811 2024-05-01  0.236  0.252  0.236  0.250      0.250    16000   MGOL

[33812 rows x 8 columns]


In [ ]:
# Find the first day when each stock has data on Yahoo Finance
min_dates = df.min(axis=0)
min_dates = min_dates[1:]  # Remove the 'Date' column
min_dates_df = pd.DataFrame(data={'Symbol': df.columns[1:], 'min_date': min_dates})
min_dates_df.set_index('Symbol', inplace=True)
min_dates_df.head()
# Generate 30 additional columns: growth_future_1d ... growth_future_30d
df = df.join(min_dates_df)
for i in range(1, 31):
    df['growth_future_{}d'.format(i)] = (df['Adj Close'] / df['Adj Close'].shift(i)) - 1
df.head()
# Find X, when the 75% quantile growth is the highest
growth_quantiles = df.describe().loc['75%', :]
growth_quantiles = growth_quantiles[['growth_future_1d', 'growth_future_2d', 'growth_future_3d', 'growth_future_4d', 'growth_future_5d', 'growth_future_6d', 'growth_future_7d', 'growth_future_8d', 'growth_future_9d', 'growth_future_10d', 'growth_future_11d', 'growth_future_12d', 'growth_future_13d', 'growth_future_14d', 'growth_future_15d', 'growth_future_16d', 'growth_future_17d', 'growth_future_18d', 'growth_future_19d', 'growth_future_20d', 'growth_future_21d', 'growth_future_22d', 'growth_future_23d', 'growth_future_24d', 'growth_future_25d', 'growth_future_26d', 'growth_future_27d', 'growth_future_28d', 'growth_future_29d', 'growth_future_30d']]
optimal_x = growth_quantiles.idxmax()
optimal_x_value = growth_quantiles.loc[optimal_x]
print(f"Optimal X: {optimal_x}")
print(f"75% quantile growth for optimal X: {optimal_x_value:.2%}")
# Addtional: You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
mean_growth = df.describe().loc['mean', :]
mean_growth = mean_growth[['growth_future_1d', 'growth_future_2d', 'growth_future_3d', 'growth_future_4d', 'growth_future_5d', 'growth_future_6d', 'growth_future_7d', 'growth_future_8d', 'growth_future_9d', 'growth_future_10d', 'growth_future_11d', 'growth_future_12d', 'growth_future_13d', 'growth_future_14d', 'growth_future_15d', 'growth_future_16d', 'growth_future_17d', 'growth_future_18d', 'growth_future_19d', 'growth_future_20d', 'growth_future_21d', 'growth_future_22d', 'growth_future_23d', 'growth_future_24d', 'growth_future_25d', 'growth_future_26d', 'growth_future_27d', 'growth_future_28d', 'growth_future_29d', 'growth_future_30d']]
median_growth = df.describe().loc['50%', :]
median_growth = median_growth[['growth_future_1d', 'growth_future_2d', 'growth_future_3d', 'growth_future_4d', 'growth_future_5d', 'growth_future_6d', 'growth_future_7d', 'growth_future_8d', 'growth_future_9d', 'growth_future_10d', 'growth_future_11d', 'growth_future_12d', 'growth_future_13d', 'growth_future_14d', 'growth_future_15d', 'growth_future_16d', 'growth_future_17d', 'growth_future_18d', 'growth_future_19d', 'growth_future_20d', 'growth_future_21d', 'growth_future_22d', 'growth_future_23d', 'growth_future_24d', 'growth_future_25d', 'growth_future_26d', 'growth_future_27d', 'growth_future_28d', 'growth_future_29d', 'growth_future_30d']]
negative_returns_count = (mean_growth < 0).sum() + (median_growth < 0).sum()
print(f"Number of negative mean and median returns: {negative_returns_count}")
# What's your recommendation: Do you suggest pursuing this strategy for an optimal X?
# The optimal X is 1, which means buying a new stock (listed on IPO) on the first day and selling it on the next day. The 75% quantile growth for this strategy is 2.18%. However, the mean and median investment returns are negative for most X values, implying that this strategy is not likely to be profitable for the majority of investors. Therefore, I would not recommend pursuing this strategy.

             Open    High     Low  Close  Adj Close    Volume   Open   High  \
Date                                                                          
2023-01-03    NaN     NaN     NaN    NaN        NaN       NaN    NaN    NaN   
2023-01-04    NaN     NaN     NaN    NaN        NaN       NaN    NaN    NaN   
2023-01-05    NaN     NaN     NaN    NaN        NaN       NaN    NaN    NaN   
2023-01-06    NaN     NaN     NaN    NaN        NaN       NaN    NaN    NaN   
2023-01-09    NaN     NaN     NaN    NaN        NaN       NaN    NaN    NaN   
...           ...     ...     ...    ...        ...       ...    ...    ...   
2024-04-25   7.61   8.700   6.860   8.34       8.34  133000.0  0.210  0.210   
2024-04-26   8.61  10.680   8.360  10.50      10.50  185500.0  0.190  0.192   
2024-04-29  10.50  12.500  10.185  11.16      11.16  213300.0  0.180  0.185   
2024-04-30  11.56  12.315   8.740   9.80       9.80  223900.0  0.170  0.171   
2024-05-01   9.85  11.060   9.720  10.34      10.34 

In [ ]:
ohlcv_df = pd.DataFrame(prices)

ValueError: If using all scalar values, you must pass an index

In [ ]:

# Drop any rows that have missing data.
ohlcv_df.dropna(inplace=True)

# Get the first day when each stock has data on Yahoo Finance.
min_dates = ohlcv_df.min(axis=0)

# Generate 30 additional columns: growth_future_1d ... growth_future_30d.
for i in range(1, 31):
    ohlcv_df['growth_future_' + str(i) + 'd'] = ohlcv_df.shift(-i)['Adj Close'] / ohlcv_df['Adj Close'] - 1

# Join the OHLCV data with the table of min_dates.
ohlcv_df = ohlcv_df.join(min_dates, how='left')

# Calculate the 75% quantile growth for each X value.
quantiles = ohlcv_df.describe(percentiles=[.75]).loc['75%']

# Find the X value with the highest 75% quantile growth.
optimal_x = quantiles.idxmax()

# Print the optimal X value and the corresponding 75% quantile growth.
print(f"Optimal X: {optimal_x}")
print(f"75% quantile growth: {quantiles[optimal_x] * 100:.2f}%")

# Calculate the mean and 50th percentile (median) investment returns for each X value.
means = ohlcv_df.mean(axis=0)
medians = ohlcv_df.median(axis=0)

# Print the mean and 50th percentile (median) investment returns for the optimal X value.
print(f"Mean investment return for optimal X: {means[optimal_x] * 100:.2f}%")
print(f"Median investment return for optimal X: {medians[optimal_x] * 100:.2f}%")

# Based on the results, it is not advisable to pursue this strategy.
# The mean and median investment returns are negative for most X values,
# implying that most investors would lose money using this strategy.

##Question 3: Is Growth Concentrated in the Largest Stocks?
Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?

Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks for 10 full years of data (2014-01-01 to 2023-12-31). You'll need to download slightly more data (7 periods before 2014-01-01 to calculate the growth_7d for the first 6 days correctly):

US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS



In [1]:
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

In [6]:
LARGEST_STOCKS = US_STOCKS  + EU_STOCKS + INDIA_STOCKS
LARGEST_STOCKS

['MSFT',
 'AAPL',
 'GOOG',
 'NVDA',
 'AMZN',
 'META',
 'BRK-B',
 'LLY',
 'AVGO',
 'V',
 'JPM',
 'NVO',
 'MC.PA',
 'ASML',
 'RMS.PA',
 'OR.PA',
 'SAP',
 'ACN',
 'TTE',
 'SIE.DE',
 'IDEXY',
 'CDI.PA',
 'RELIANCE.NS',
 'TCS.NS',
 'HDB',
 'BHARTIARTL.NS',
 'IBN',
 'SBIN.NS',
 'LICI.NS',
 'INFY',
 'ITC.NS',
 'HINDUNILVR.NS',
 'LT.NS']

##Now let's add the top 12-22 stocks (as of end-April 2024):

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

You should be able to obtain stats for 33 LARGEST STOCKS and 32 LARGE STOCKS (from the actual stats on Yahoo Finance)


In [7]:
NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

In [16]:
import time

largest_stocks_df = pd.DataFrame({'A' : []})
# Define the start and end dates
start_date = '2013-12-25'
end_date = '2024-01-01'
for i,ticker in enumerate(LARGEST_STOCKS):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,start=start_date, end=end_date,
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  historyPrices['growth_7d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(7)



  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if largest_stocks_df.empty:
    largest_stocks_df = historyPrices
  else:
    largest_stocks_df = pd.concat([largest_stocks_df, historyPrices], ignore_index=True)

[*********************100%%**********************]  1 of 1 completed

0 MSFT



[*********************100%%**********************]  1 of 1 completed

1 AAPL



[*********************100%%**********************]  1 of 1 completed

...

30 ITC.NS



[*********************100%%**********************]  1 of 1 completed

31 HINDUNILVR.NS



[*********************100%%**********************]  1 of 1 completed

32 LT.NS


In [17]:
large_stocks_df = pd.DataFrame({'A' : []})
# Define the start and end dates
start_date = '2013-12-25'
end_date = '2024-04-30'
for i,ticker in enumerate(LARGEST_STOCKS):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,start=start_date, end=end_date,
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  historyPrices['growth_7d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(7)



  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if large_stocks_df.empty:
    large_stocks_df = historyPrices
  else:
    large_stocks_df = pd.concat([large_stocks_df, historyPrices], ignore_index=True)

0 MSFT


[*********************100%%**********************]  1 of 1 completed


1 AAPL


[*********************100%%**********************]  1 of 1 completed


2 GOOG


...

[*********************100%%**********************]  1 of 1 completed


31 HINDUNILVR.NS


[*********************100%%**********************]  1 of 1 completed


32 LT.NS


[*********************100%%**********************]  1 of 1 completed


##Calculate growth_7d for every stock and every day. Get the average daily growth_7d for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

For example, for the first of data you should have:

Date	ticker_category	growth_7d
2014-01-01	LARGE	1.011684
2014-01-01	LARGEST	1.011797
On that day, the LARGEST group was growing faster than LARGE one (new stocks).

Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value). For example, if you find that 1700 out of 2595 days meet this condition, it means that 1700/2595 = 0.655, or approximately 66% of days, the LARGE stocks were growing faster than the LARGEST ones. This suggests that you should consider extending your dataset with more stocks to seek higher growth.

HINT: you can use pandas.pivot_table() to "flatten" the table (LARGE and LARGEST growth_7d as columns)

In [19]:
# prompt: Calculate growth_7d for every stock and every day. Get the average daily growth_7d for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

import pandas as pd
# Merge the two DataFrames
merged_df = pd.merge(large_stocks_df, largest_stocks_df, on=['Date', 'Ticker'])

# Rename the columns for clarity
merged_df.rename(columns={'growth_7d_x': 'growth_7d_large', 'growth_7d_y': 'growth_7d_largest'}, inplace=True)

# Create a pivot table to "flatten" the table
pivot_table = pd.pivot_table(merged_df, values=['growth_7d_large', 'growth_7d_largest'], index='Date', columns='Ticker')





In [23]:
merged_df

,Open_x,High_x,Low_x,Close_x,Adj Close_x,Volume_x,Ticker,Year_x,Month_x,Weekday_x,...,Open_y,High_y,Low_y,Close_y,Adj Close_y,Volume_y,Year_y,Month_y,Weekday_y,growth_7d_largest
0,37.200001,37.490002,37.169998,37.439999,31.468403,17612800,MSFT,2013,12,3,...,37.200001,37.490002,37.169998,37.439999,31.468407,17612800,2013,12,3,NaN
1,37.580002,37.619999,37.169998,37.290001,31.342325,14563000,MSFT,2013,12,4,...,37.580002,37.619999,37.169998,37.290001,31.342327,14563000,2013,12,4,NaN
2,37.220001,37.380001,36.900002,37.290001,31.342325,16290500,MSFT,2013,12,0,...,37.220001,37.380001,36.900002,37.290001,31.342327,16290500,2013,12,0,NaN
3,37.400002,37.580002,37.220001,37.410000,31.443199,17503500,MSFT,2013,12,1,...,37.400002,37.580002,37.220001,37.410000,31.443201,17503500,2013,12,1,NaN
4,37.349998,37.400002,37.099998,37.160000,31.233067,30632200,MSFT,2014,1,3,...,37.349998,37.400002,37.099998,37.160000,31.233067,30632200,2014,1,3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80879,3424.000000,3496.000000,3408.600098,3477.949951,3477.949951,1681707,LT.NS,2023,12,4,...,3424.000000,3496.000000,3408.600098,3477.949951,3477.949951,1681707,2023,12,4,1.022987
80880,3477.949951,3508.350098,3477.949951,3490.050049,3490.050049,1072263,LT.NS,2023,12,1,...,3477.949951,3508.350098,3477.949951,3490.050049,3490.050049,1072263,2023,12,1,1.016588
80881,3510.000000,3549.000000,3504.149902,3544.000000,3544.000000,1389266,LT.NS,2023,12,2,...,3510.000000,3549.000000,3504.149902,3544.000000,3544.000000,1389266,2023,12,2,1.016055
80882,3545.000000,3559.949951,3500.500000,3518.050049,3518.050049,3371121,LT.NS,2023,12,3,...,3545.000000,3559.949951,3500.500000,3518.050049,3518.050049,3371121,2023,12,3,1.007575
